### 모델링을 위한 기본 전처리

- 기본적인 전처리는 파이썬에서 가능한 부분은 파이썬에서 수행하며, SQL에서 가능한 부분은 SQL에서 수행한다.


- 다루는 내용
    - `시간 관련 전처리`
        - DATETIME에서 MONTH, HOUR, WEEKDAY 등 추출
    - `BigQuery GIS`로 Reverse Geocoding
        - 좌표 => zip_code로 변환
    - `학습/ 테스트 데이터 분리`
        - train_split_test
    - `카테고리컬` 데이터 전처리
        - One Hot Encoding
        - Label Encoding

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import warnings
import seaborn as sns
from sklearn.linear_model import LinearRegression
warnings.filterwarnings('ignore')

PROJECT_ID = 'new-york-taxi-309501'

### 시간관련 전처리
- Bigquery vs Python
    - Bigquery
        - DATETIME_TRUNC 함수를 사용하면 데이터 자르기 가능
        - EXTRACT 함수를 사용하면 MONTH, WEEK, DAY, HOUR등을 쉽게 추출 가능
        - FORMAT_DATETIME 함수를 사용하면 요일 추출 가능( 1이 월요일, Pandas에서는 0이 월요일 )

In [2]:
extract_query = """
SELECT
    *,
    EXTRACT(MONTH FROM pickup_hour) AS month,
    EXTRACT(DAY FROM pickup_hour) AS day,
    CAST(format_datetime('%u', pickup_hour) AS INT64) -1 AS weekday, # Python과 맞추기 위해 -1
    EXTRACT(HOUR FROM pickup_hour) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_hour) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend
FROM (
    SELECT 
        DATETIME_TRUNC(pickup_datetime, hour) AS pickup_hour,
        count(*) AS cnt
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015` 
    WHERE EXTRACT(MONTH from pickup_datetime) = 1
    GROUP BY pickup_hour
)
ORDER BY pickup_hour
"""

df = pd.read_gbq(query=extract_query, dialect='standard', project_id='new-york-taxi-309501')

Downloading: 100%|███████████████████████████████████████████████████████████████| 744/744 [00:00<00:00, 1881.15rows/s]


In [3]:
df

,pickup_hour,cnt,month,day,weekday,hour,is_weekend
0,2015-01-01 00:00:00,28312,1,1,3,0,0
1,2015-01-01 01:00:00,31707,1,1,3,1,0
2,2015-01-01 02:00:00,28068,1,1,3,2,0
3,2015-01-01 03:00:00,24288,1,1,3,3,0
4,2015-01-01 04:00:00,17081,1,1,3,4,0
...,...,...,...,...,...,...,...
739,2015-01-31 19:00:00,32436,1,31,5,19,1
740,2015-01-31 20:00:00,27555,1,31,5,20,1
741,2015-01-31 21:00:00,27477,1,31,5,21,1
742,2015-01-31 22:00:00,29862,1,31,5,22,1


- Python 
    - Pandas datetime 형식으로 처리하기
    - `pd.to_datetime`으로 datetime 변환 후, df[col].dt.xxx 로 추출

In [4]:
extract_query = """
SELECT 
    DATETIME_TRUNC(pickup_datetime, hour) AS pickup_hour,
    count(*) AS cnt
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015` 
WHERE EXTRACT(MONTH from pickup_datetime) = 1
GROUP BY pickup_hour
ORDER BY pickup_hour
"""

df = pd.read_gbq(query=extract_query, dialect='standard', project_id='new-york-taxi-309501')

Downloading: 100%|████████████████████████████████████████████████████████████████| 744/744 [00:00<00:00, 791.05rows/s]


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 2 columns):
pickup_hour    744 non-null datetime64[ns]
cnt            744 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 11.8 KB


In [7]:
df['month'] = df['pickup_hour'].dt.month
df['day'] = df['pickup_hour'].dt.day
df['weekday'] = df['pickup_hour'].dt.weekday
df['hour'] = df['pickup_hour'].dt.hour
df['is_weekend'] = (df['pickup_hour'].dt.weekday // 5 == 1).astype(int)

In [8]:
df.head()

,pickup_hour,cnt,month,day,weekday,hour,is_weekend
0,2015-01-01 00:00:00,28312,1,1,3,0,0
1,2015-01-01 01:00:00,31707,1,1,3,1,0
2,2015-01-01 02:00:00,28068,1,1,3,2,0
3,2015-01-01 03:00:00,24288,1,1,3,3,0
4,2015-01-01 04:00:00,17081,1,1,3,4,0


### `BigQuery GIS`로 Reverse Geocoding
- 좌표 => zip_code로 변환
- ST_CONTAINS 함수로 추출 가능

In [9]:
base_query = """
WITH base_data AS 
(
  SELECT nyc_taxi.*, gis.* EXCEPT (zip_code_geom)
  FROM (
    SELECT *
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
    WHERE 
        EXTRACT(MONTH from pickup_datetime) = 1
        and pickup_latitude  <= 90 and pickup_latitude >= -90
    ) AS nyc_taxi
  JOIN (
    SELECT zip_code, state_code, state_name, city, county, zip_code_geom
    FROM `bigquery-public-data.geo_us_boundaries.zip_codes`
    WHERE state_code='NY'
    ) AS gis 
  ON ST_CONTAINS(zip_code_geom, st_geogpoint(pickup_longitude, pickup_latitude))
)

SELECT 
    zip_code,
    DATETIME_TRUNC(pickup_datetime, hour) as pickup_hour,
    EXTRACT(MONTH FROM pickup_datetime) AS month,
    EXTRACT(DAY FROM pickup_datetime) AS day,
    CAST(format_datetime('%u', pickup_datetime) AS INT64) -1 AS weekday,
    EXTRACT(HOUR FROM pickup_datetime) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend,
    COUNT(*) AS cnt
FROM base_data 
GROUP BY zip_code, pickup_hour, month, day, weekday, hour, is_weekend
ORDER BY pickup_hour


"""

base_df = pd.read_gbq(query=base_query, dialect='standard', project_id='new-york-taxi-309501')

Downloading: 100%|██████████████████████████████████████████████████████████| 87020/87020 [00:05<00:00, 16684.22rows/s]


In [10]:
base_df

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt
0,10037,2015-01-01 00:00:00,1,1,3,0,0,26
1,10039,2015-01-01 00:00:00,1,1,3,0,0,9
2,11105,2015-01-01 00:00:00,1,1,3,0,0,32
3,11238,2015-01-01 00:00:00,1,1,3,0,0,95
4,11203,2015-01-01 00:00:00,1,1,3,0,0,3
...,...,...,...,...,...,...,...,...
87015,10044,2015-01-31 23:00:00,1,31,5,23,1,6
87016,10009,2015-01-31 23:00:00,1,31,5,23,1,1166
87017,10020,2015-01-31 23:00:00,1,31,5,23,1,66
87018,11102,2015-01-31 23:00:00,1,31,5,23,1,23
